We'll at least start out with the given guys before feature engineering.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.feature_selection import RFECV #was hoping for this without the reverse, but I don't see it. Oh well when
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV
#it's just 8; well, it'll be more hectic momentarily...

from sklearn.metrics import make_scorer
from sklearn.metrics import root_mean_squared_log_error #Phew this on is on there... I mean find if we'd have to square,
#but rather not have to write the formula.

from sklearn.metrics import mean_squared_log_error

# from sklearn.metrics import make_scorer

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR

In [4]:
from sklearn.preprocessing import PolynomialFeatures

In [62]:
#The following is in case I need it later. As, it seems quite barbaric to sort through the list each one... however this would require
#changing the numbers into a dataframe.

def log_prepper(num):
    return min(0, num)

In [25]:
#Actual custom rmsle....

#Ideally, I'd rather not have to actually write the formula out... 
#Unfortunately this didn't work.Unsure what I did wrong,but eh...

def cus_rmsle(actuals, preds):
    rmsle = 0
    for i in range(len(actuals)):
        rmsle += np.log(1+min(preds[i], 0)) - np.log(1+actuals[i])
    rmsle = rmsle/len(preds) #Hmm, /+?
    return rmsle**.5

In [6]:
#Yeah, after thinking about things the scoring should affect patterns detected, so instead of r2 let's finally change things...

scoring = make_scorer(root_mean_squared_log_error, greater_is_better=False)
# scoring = make_scorer(mean_squared_log_error, greater_is_better=False, squared=False)

In [5]:
df = pd.read_csv('../data/cleaned_train.csv')

df = pd.get_dummies(df, columns=['sex'], dtype=int, drop_first=True)
#I honestly forgot to earlier, although I do like the data is preserved: At least at the moment we also need to dummify sex
#to get our lovely sklearn methods to work:

print(df.shape)
df.head()#Note that the baseline, ie 0s in the rest, will be indicativ of female abalones.

(90609, 10)


,length,diameter,height,whole_weight,whole_weight1,whole_weight2,shell_weight,rings,sex_I,sex_M
0,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11,0,0
1,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11,0,0
2,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6,1,0
3,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10,0,1
4,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9,1,0


In [15]:
X = df.drop('rings', axis=1)
y = df['rings']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=26) #Relatively large test size as we
#have so much data to work with...
#Re. 26 - I like that number. Discussion not for now

mms = MinMaxScaler() #Temp. removed to have an easier time seeing which feature names were used.
X_train = mms.fit_transform(X_train)
X_test = mms.transform(X_test)

Given that this is somewhat of a professional environment, as opposed to my boot camp I just finished, I won't bother to explain all of my thoughts. However, note that I'll reuse this and similar lines of code until finding desired results.

In [6]:
# model_defaults.get_feature_names_out()

#Hmm, quite interesting that we chose all the weights.... ie I'd understand if one, but
#apparetly the relative discremencies upon focusing on less and less of the alabone, quite literally, is that useful.

#Also, the distinguishment between female is somehwat negligible, despite hte many difference we obsered... however from
#female to on-binary remains significant.

### Temp:

I'm going back and trying to get the custom scorer going... it finally is clear to me that the issue I had then is what I had a while back and what I have no (which in short means): Seemingly those records whose predictions would be lower than 0 cause issues even in the fitting stage... Which makes sense as we're already taking the rmsle in the get-go, using htat to try to determine the model... likely can get around it in the linear regression stage, but to use rfe.... and for forests...

So, it is incumbent upon us to get around this.

### Conclusion re. Temp:

Ironic the name of hte Stackoverflow user that commented this - StupidWolf: Anyways, to focus on doing log transformations also to the target. So, I went ahead and did that on modeling_ii and it worked... Ie once the target is also log scaled then it's basically not going to happen to ever get a negative prediction... so we got this!

In [29]:
#Oh, just remembered we're supposed to use MSLE. At th least we have a benchmark of what's good 'objectively'

lr = LinearRegression()
rfe = RFECV(lr, step=1, cv=5
            ,min_features_to_select=3)

model_defaults = rfe.fit(X_train, y_train)

preds_model_defaults = rfe.predict(X_test)#And we're somehow getting some negative predictions.... note that againt his is
#without any feature engineering.... Regardless, we'll just brute force it. Unsure if we have a better wayt han the following:

for i in range(len(preds_model_defaults)):
    if preds_model_defaults[i] < 0:
        preds_model_defaults[i] = 0
print(preds_model_defaults[preds_model_defaults<0])

print(model_defaults.score(X_train, y_train), model_defaults.score(X_test, y_test))
print(model_defaults.cv_results_['mean_test_score'])
root_mean_squared_log_error(y_test, preds_model_defaults)

C:\Users\bwolf\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_regression.py:666: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared logarithmic error, use the function'root_mean_squared_log_error'.
  warnings.warn(


ValueError: Root Mean Squared Logarithmic Error cannot be used when targets contain negative values.

In [ ]:
# []
# 0.6062234959016377 0.6034787674182922
# [0.59003897 0.59367525 0.59568496 0.59954274 0.59970396 0.60580828
#  0.60580744]
# 0.16508650876858538

In [8]:
#A little tempted to play around with the defaults, but eh...consideing that we likely need to engineer some things...

rf = RandomForestRegressor(random_state=26)

##Seems a bit redundant to use both here...
# rfe = RFECV(rf, step=1, cv=5, 
#                        min_features_to_select=3)

model_defaults = rf.fit(X_train, y_train)

preds_model_defaults = rf.predict(X_test)

for i in range(len(preds_model_defaults)):
    if preds_model_defaults[i] < 0:
        preds_model_defaults[i] = 0

print(model_defaults.score(X_train, y_train), model_defaults.score(X_test, y_test))
# print(cross_val_score(rf, 
#                       X_train, y_train, cv = 5).mean()) #Yeah,c ommenting this guy out... taking too long plus a bit redundant
# #Given that random forests are already aggregativing everything
root_mean_squared_log_error(y_test, preds_model_defaults)

#oooooh yesss this is quite juicy... quite juicy abalane 

0.9504620112507841 0.6557858728716266


0.15345037771109488

In [23]:
bag = BaggingRegressor(random_state=26)

##Seems a bit redundant to use both here...
# rfe = RFECV(rf, step=1, cv=5, 
#                        min_features_to_select=3)

model_defaults = bag.fit(X_train, y_train)

preds_model_defaults = bag.predict(X_test)

for i in range(len(preds_model_defaults)):
    if preds_model_defaults[i] < 0:
        preds_model_defaults[i] = 0

print(model_defaults.score(X_train, y_train), model_defaults.score(X_test, y_test))
# print(cross_val_score(rf, 
#                       X_train, y_train, cv = 5).mean()) #Yeah,c ommenting this guy out... taking too long plus a bit redundant
# #Given that random forests are already aggregativing everything
root_mean_squared_log_error(y_test, preds_model_defaults)

0.9313299951392628 0.6238598931844348


0.16064598506264355

In [20]:
# pipe = Pipeline([
#     ('svr', SVR())
# ])
# pipe_params = {'svr__kernel' : ['rbf'] #Eh, taking too long to do all of them... just go with rbf for now
#                ,'svr__C' : [.9, .95 , 1]
#               }
# gs = GridSearchCV(pipe,
#                   param_grid=pipe_params,
#                   cv=5)

# gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Pipeline(steps=[('svr', SVR())]),
             param_grid={'svr__C': [0.9, 0.95, 1], 'svr__kernel': ['rbf']})

In [21]:
# preds_model_defaults = rf.predict(X_test)
# for i in range(len(preds_model_defaults)):
#     if preds_model_defaults[i] < 0:
#         preds_model_defaults[i] = 0

# print(gs.score(X_train, y_train), gs.score(X_test, y_test))
# print(gs.best_score_)
# print(gs.best_params_)
        
# root_mean_squared_log_error(y_test, preds_model_defaults)

##0.6187870521022931 0.6175574607103471 #Commenting out as it took way too long... note that it has the same RMSLE as forests
## 0.6167895215540445
## {'svr__C': 1, 'svr__kernel': 'rbf'}
## 0.15345037771109488

0.6187870521022931 0.6175574607103471
0.6167895215540445
{'svr__C': 1, 'svr__kernel': 'rbf'}


0.15345037771109488

So, bottom line we 'like' the accuracy and processing time of Random Forests. Perhaps regression is just that intense, or I'm underappeicating how giant this dataset is, but SVR took forever. So, likely we'll jump just do Forests as we move on. We'll see.

## The Start of Feature Engineering: Polynomials

We'll keep things PG and just go up to squared terms, naturally considering interaction effects as well.

In [64]:
X = df.drop('rings', axis=1)
y = df['rings']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=26)

poly = PolynomialFeatures(degree = 2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

df_poly_train = pd.DataFrame(X_train_poly, columns=poly.get_feature_names_out(X.columns)) #We'll use this later
df_poly_test = pd.DataFrame(X_test_poly, columns=poly.get_feature_names_out(X.columns))

mms = MinMaxScaler()
X_train = mms.fit_transform(X_train)
X_test = mms.transform(X_test)

In [11]:
len(poly.get_feature_names_out(X.columns))

54

In [36]:
#Hmm, so a maximum of 54 features now. let's do this!

lr = LinearRegression()
rfe = RFECV(lr, step=1, cv=5,
                       min_features_to_select=35)

model_defaults = rfe.fit(X_train_poly, y_train)

preds_model_defaults = rfe.predict(X_test_poly)#And we're somehow getting some negative predictions.... note that againt his is
#without any feature engineering.... Regardless, we'll just brute force it. Unsure if we have a better wayt han the following:

for i in range(len(preds_model_defaults)):
    if preds_model_defaults[i] < 0:
        preds_model_defaults[i] = 0
print(preds_model_defaults[preds_model_defaults<0])

print(model_defaults.score(X_train_poly, y_train), model_defaults.score(X_test_poly, y_test))
print(model_defaults.cv_results_['mean_test_score'])
root_mean_squared_log_error(y_test, preds_model_defaults)

[]
0.637502643156102 0.6391210169168662
[0.61664033 0.61906504 0.62020718 0.62345992 0.62343427 0.62558104
 0.62686518 0.6280259  0.62854671 0.62875105 0.62839985 0.62840971
 0.6287009  0.62930472 0.62911993 0.62926181 0.62897665 0.62941128
 0.62974946 0.62969177 0.6300942  0.63044306 0.63204007 0.63281593
 0.6332747  0.63349061 0.6335524  0.63323694 0.633351   0.633187
 0.63318795 0.63320335 0.63326952 0.63328497 0.63329516 0.63333263
 0.63334052 0.63339293 0.63334014 0.6333579  0.63336914 0.63336319
 0.63336625 0.63336738 0.63336738]


0.1562207342973044

In [44]:
print(model_defaults.n_features_)
model_defaults.get_feature_names_out()

36


array(['x0', 'x1', 'x2', 'x3', 'x4', 'x6', 'x9', 'x10', 'x11', 'x12',
       'x13', 'x14', 'x15', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23',
       'x26', 'x28', 'x29', 'x30', 'x31', 'x34', 'x35', 'x36', 'x39',
       'x40', 'x41', 'x42', 'x44', 'x45', 'x46', 'x48', 'x49'],
      dtype=object)

In [58]:
#Likely some better method exists, but eh...

targets = []
for i in model_defaults.get_feature_names_out():
    targets.append(int(i[1:]))
target_columns = []
for i in targets:
    target_columns.append(df_poly_train.columns[i])

In [60]:
target_columns[:6] #Hmm, so on a pure scale we are disregarding the second weight as well as pure sex variables. Howeve,r htey
#do manifest in conjecture with others.

#Recall earlier how we proposed that inherently, likely, the sex is meaningless and just a proxy for expected weights. So,
#perhaps this is inline with what we assumed.

['length',
 'diameter',
 'height',
 'whole_weight',
 'whole_weight1',
 'shell_weight']

Recall that the pure linear rmsle with lr was 0.16508650876858538.

Hmm, originally I'm doing this after min/max scaling...yeah that's probably not a move as perhaps the scaling is interfering with their effects. Oh, while all that is true I had a more fundamental mistake - NOOB remember to score around the POLY dataset and not the original one...

There we go, at 50 we already have a much lower RMSLE of .156. Let's keep on going for a while.

In [ ]:
#I'll leave the name the same, so note how now it's just the subset per the earlier linear regression.

X_train_poly = np.array(df_poly_train[target_columns])
X_test_poly = np.array(df_poly_test[target_columns])

In [77]:
rf = RandomForestRegressor(random_state=26)

model_defaults = rf.fit(X_train_poly, y_train)

preds_model_defaults = rf.predict(X_test_poly)

for i in range(len(preds_model_defaults)):
    if preds_model_defaults[i] < 0:
        preds_model_defaults[i] = 0

print(model_defaults.score(X_train_poly, y_train), model_defaults.score(X_test_poly, y_test))
# print(cross_val_score(rf, 
#                       X_train, y_train, cv = 5).mean()) #Yeah,c ommenting this guy out... taking too long plus a bit redundant
# #Given that random forests are already aggregativing everything
root_mean_squared_log_error(y_test, preds_model_defaults)

#A fair bit lower... Hmm. My intuition tells me to even keep on going for now and get to logs first. So, perhaps
#we'll return and actually tune the forest's parameters.

#Tempted to do SVR just cause I need to be afk for a while...yeah why not. Let's do it.

0.9500471760962943 0.6566633589144847


0.15307428574200113

In [ ]:
#Potential future spot for bagged forests x polys.

In [75]:
X_train_poly = np.array(df_poly_train[target_columns])
X_test_poly = np.array(df_poly_test[target_columns])

(22653, 36)

In [76]:
svr = SVR() #Going with defautls at least for now

svr = svr.fit(X_train_poly, y_train)

preds_model_defaults = svr.predict(X_test_poly)
for i in range(len(preds_model_defaults)):
    if preds_model_defaults[i] < 0:
        preds_model_defaults[i] = 0

print(svr.score(X_train_poly, y_train), svr.score(X_test_poly, y_test))
        
root_mean_squared_log_error(y_test, preds_model_defaults)

#Aww, all that waiting and not even better than the forests.

0.6256051790865003 0.6257469825562314


0.1549904012357784

So, we'e made a bit of progress with introducing polynomial degree 2 and interaction effects. While RF is still our best and preferred model, we're down from .15345 to .15307 RMSLE.

## Round I.5

I was on the sofa, relaxing with some William James (ironic that this happeneded in his section on 'Time'), and I recalled that it behooves me to also check if perhaps the features, whether linear or polynomial, possess a stronger/more accurate relationship with rings when rings are squared.

In [61]:
#Unless I find particular succuess in this, I'll just do linear and forests

X = df.drop('rings', axis=1)
y = df['rings']**2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=26)

poly = PolynomialFeatures(degree = 2, include_bias=False)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

df_poly_train = pd.DataFrame(X_train_poly, columns=poly.get_feature_names_out(X.columns)) #We'll use this later
df_poly_test = pd.DataFrame(X_test_poly, columns=poly.get_feature_names_out(X.columns))

mms = MinMaxScaler()
X_train = mms.fit_transform(X_train)
X_test = mms.transform(X_test)

In [9]:
lr = LinearRegression()
rfe = RFECV(lr, step=1, cv=5,
                       min_features_to_select=30)

model_defaults = rfe.fit(X_train_poly, y_train)

preds_model_defaults = rfe.predict(X_test_poly)

for i in range(len(preds_model_defaults)):
    if preds_model_defaults[i] < 0:
        preds_model_defaults[i] = 0
print(preds_model_defaults[preds_model_defaults<0])

print(model_defaults.score(X_train_poly, y_train), model_defaults.score(X_test_poly, y_test))
print(model_defaults.cv_results_['mean_test_score'])
root_mean_squared_log_error(y_test, preds_model_defaults)

#Oh, that's much worse... well, I ain't given' up yet!

[]
0.5362205789508885 0.535225347225045
[0.53084338 0.53122091 0.53112899 0.53113058 0.53160492 0.53171323
 0.53171953 0.5314713  0.53149092 0.53148084 0.53145503 0.53145165
 0.53152492 0.53152079 0.53152905 0.53151887 0.53163101 0.53163282
 0.53164606 0.53162944 0.53151049 0.53152383 0.53151271 0.53152181
 0.53152181]


0.36388271962439356

In [12]:
print(model_defaults.n_features_)

targets = []
for i in model_defaults.get_feature_names_out():
    targets.append(int(i[1:]))
target_columns = []
for i in targets:
    target_columns.append(df_poly_train.columns[i])
    
X_train_poly = np.array(df_poly_train[target_columns])
X_test_poly = np.array(df_poly_test[target_columns])

36


In [13]:
rf = RandomForestRegressor(random_state=26)

model_defaults = rf.fit(X_train_poly, y_train)

preds_model_defaults = rf.predict(X_test_poly)

for i in range(len(preds_model_defaults)):
    if preds_model_defaults[i] < 0:
        preds_model_defaults[i] = 0

print(model_defaults.score(X_train_poly, y_train), model_defaults.score(X_test_poly, y_test))
root_mean_squared_log_error(y_test, preds_model_defaults)

#Slightly better than linreg, but eh... still much worse than when assumng rings are linear. Worth a shot though.

0.9353330249126093 0.5540950426000051


0.34327315748689535

## Round II - Logs

As we predicted we'd have to do, given assumptions in using a log based metric (first time using such a thing so I honestly don't know if it's grounded anywhere. We'd do it anyways, but still...)

In [7]:
#Hmm, we'll start off assuming a simple log of everything. Unsure if that would also include rings...

df_log = np.log(df)

df_log['sex_I'] = df['sex_I']
df_log['sex_M'] = df['sex_M']
#Recall that 0 is e to the power of negative infinity; 1 is  to the power of 0... since htese are dummies
#let's revert them back to their originals

df_log.head()

C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\internals\blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


,length,diameter,height,whole_weight,whole_weight1,whole_weight2,shell_weight,rings,sex_I,sex_M
0,-0.597837,-0.843970,-1.897120,-0.259419,-1.113218,-1.920730,-1.427116,2.397895,0,0
1,-0.462035,-0.713350,-1.931022,0.122218,-0.780886,-1.285544,-1.139434,2.397895,0,0
2,-1.832581,-2.207275,-3.688879,-3.863233,-5.203007,-5.809143,-5.298317,1.791759,1,0
3,-0.519194,-0.744440,-1.897120,-0.089378,-0.979497,-1.582309,-1.386294,2.302585,0,1
4,-0.588787,-0.855666,-2.040221,-0.245901,-0.995605,-1.832581,-1.622017,2.197225,1,0


In [8]:
X = df_log.drop('rings', axis=1)
y = df_log['rings']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=26)

mms = MinMaxScaler()
X_train = mms.fit_transform(X_train)
X_test = mms.transform(X_test)

In [9]:
lr = LinearRegression()

#A bit superfl. to have rfe here, but you neverknow... Plus makes the rest of the coding easier.
rfe = RFECV(lr, step=1, cv=5,
                       min_features_to_select=5)

model_defaults = rfe.fit(X_train, y_train)

preds_model_defaults = model_defaults.predict(X_test)

for i in range(len(preds_model_defaults)):
    if preds_model_defaults[i] < 0:
        preds_model_defaults[i] = 0
print(preds_model_defaults[preds_model_defaults<0])

print(model_defaults.score(X_train, y_train), model_defaults.score(X_test, y_test))
print(model_defaults.cv_results_['mean_test_score'])
root_mean_squared_log_error(y_test, preds_model_defaults)

#Awww yeah, that's awesome!

[]
0.6777817153167525 0.6834449547047344
[0.67310332 0.67314658 0.67675627 0.67757212 0.67759681]


0.05848349109552299

In [11]:
model_defaults.ranking_

array([1, 1, 1, 1, 1, 1, 1, 1, 1])

In [23]:
# print(model_defaults.n_features_)

# targets = []
# for i in model_defaults.get_feature_names_out():
#     targets.append(int(i[1:]))
# target_columns = []
# for i in targets:
#     target_columns.append(df_poly_train.columns[i])
    
# X_train = np.array(df_poly_train[target_columns])
# X_test = np.array(df_poly_test[target_columns])

9


In [12]:
rf = RandomForestRegressor(random_state=26)

model_defaults = rf.fit(X_train, y_train)

preds_model_defaults = model_defaults.predict(X_test)

for i in range(len(preds_model_defaults)):
    if preds_model_defaults[i] < 0:
        preds_model_defaults[i] = 0

print(model_defaults.score(X_train, y_train), model_defaults.score(X_test, y_test))
root_mean_squared_log_error(y_test, preds_model_defaults)

#Nice, a bit better.

0.9595105586184751 0.7214613214990553


0.05449082576242973

In [8]:
#While I'm here I'll go ahead and do non-logged rings:
X = df_log.drop('rings', axis=1)
y = df['rings']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=26)

mms = MinMaxScaler()
X_train = mms.fit_transform(X_train)
X_test = mms.transform(X_test)

In [9]:
lr = LinearRegression()

#A bit superfl. to have rfe here, but you neverknow... Plus makes the rest of the coding easier.
rfe = RFECV(lr, step=1, cv=5,
                       min_features_to_select=5)

model_defaults = rfe.fit(X_train, y_train)

preds_model_defaults = model_defaults.predict(X_test)

for i in range(len(preds_model_defaults)):
    if preds_model_defaults[i] < 0:
        preds_model_defaults[i] = 0
print(preds_model_defaults[preds_model_defaults<0])

print(model_defaults.score(X_train, y_train), model_defaults.score(X_test, y_test))
print(model_defaults.cv_results_['mean_test_score'])
print(model_defaults.n_features_)
root_mean_squared_log_error(y_test, preds_model_defaults) #Yeah, not nearly as high... Perhaps it's just me being stubborn,
#however I don't see why it should inherently be necessary to also natural log the target [just recall that then
#the targets, now logged, are targetting the 'default' rings and not logged rings]

[]
0.5741896897846039 0.5765323834122111
[0.56682665 0.56683724 0.5670822  0.57376272 0.57390427]
9


0.20175851656732902

## Wrapping Up

Although much more analysis could be done - ie let's see if perhaps rings shouldn't be natural logged too or perhaps certain features should be polynomialed... this is good enough. Inded, if I'm using the right metric then this blows everybody off the map...

In [50]:
submission_model = model_defaults #Scruff when this might get changed a lot... oh well.

In [64]:
df = pd.read_csv('../data/cleaned_test.csv')
df = pd.get_dummies(df, columns=['sex'], dtype=int, drop_first=True)
print(df.shape)
df.head()

(60411, 10)


,id,length,diameter,height,whole_weight,whole_weight1,whole_weight2,shell_weight,sex_I,sex_M
0,90615,0.645,0.475,0.155,1.2380,0.6185,0.3125,0.3005,0,1
1,90616,0.580,0.460,0.160,0.9830,0.4785,0.2195,0.2750,0,1
2,90617,0.560,0.420,0.140,0.8395,0.3525,0.1845,0.2405,0,1
3,90618,0.570,0.490,0.145,0.8740,0.3525,0.1865,0.2350,0,1
4,90619,0.415,0.325,0.110,0.3580,0.1575,0.0670,0.1050,1,0


In [65]:
df_log = np.log(df)

df_log['sex_I'] = df['sex_I']
df_log['sex_M'] = df['sex_M']
df_log['id'] = df['id']

df_log.head()

C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\internals\blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


,id,length,diameter,height,whole_weight,whole_weight1,whole_weight2,shell_weight,sex_I,sex_M
0,90615,-0.438505,-0.744440,-1.864330,0.213497,-0.480458,-1.163151,-1.202308,0,1
1,90616,-0.544727,-0.776529,-1.832581,-0.017146,-0.737099,-1.516403,-1.290984,0,1
2,90617,-0.579818,-0.867501,-1.966113,-0.174949,-1.042705,-1.690106,-1.425035,0,1
3,90618,-0.562119,-0.713350,-1.931022,-0.134675,-1.042705,-1.679324,-1.448170,0,1
4,90619,-0.879477,-1.123930,-2.207275,-1.027222,-1.848330,-2.703063,-2.253795,1,0


In [56]:
df_log.describe()

,id,length,diameter,height,whole_weight,whole_weight1,whole_weight2,shell_weight,sex_I,sex_M
count,60411.000000,60411.000000,60411.000000,6.041100e+04,60411.000000,60411.000000,60411.000000,60411.000000,60411.000000,60411.000000
mean,120820.000000,-0.691101,-0.948531,-inf,-0.487624,-1.339299,-2.037252,-1.731688,0.368161,0.344027
std,17439.297893,0.270962,0.292039,NaN,0.840978,0.863855,0.855001,0.827873,0.482309,0.475054
min,90615.000000,-2.590267,-2.900422,-inf,-6.214608,-6.907755,-7.600902,-6.502290,0.000000,0.000000
25%,105717.500000,-0.798508,-1.064211,-2.207275e+00,-0.868692,-1.723167,-2.447611,-2.120264,0.000000,0.000000
50%,120820.000000,-0.606969,-0.855666,-1.966113e+00,-0.221270,-1.111698,-1.795767,-1.491655,0.000000,0.000000
75%,135922.500000,-0.510826,-0.755023,-1.832581e+00,0.067659,-0.766794,-1.458865,-1.187444,1.000000,1.000000
max,151025.000000,-0.223144,-0.430783,9.075436e-02,1.038685,0.397433,-0.443946,0.003992,1.000000,1.000000


In [66]:
X = df_log.drop('id', axis=1) #Eh,t his bit seems a bit scruff to me, but eh...

X_val = mms.transform(X)

# preds = submission_model.predict(X_val)

In [77]:
submission = pd.DataFrame(df['id'])
submission['Rings'] = np.exp(submission_model.predict(X_val))
submission.head()

,id,Rings
0,90615,10.316769
1,90616,9.571289
2,90617,10.187230
3,90618,11.315985
4,90619,7.552523


In [75]:
# submission['Rings'] = submission['Rings'].round(0) #Might as well round... these guys are whole numbers so... pragmatically
# #might be better to not bother, as we'll be going up and down... oh well. Let's see what happens
# submission.head()

,id,Rings
0,90615,10.0
1,90616,10.0
2,90617,10.0
3,90618,11.0
4,90619,8.0


In [78]:
submission.to_csv(
    '../data/submission_ii.csv', index=False)

## Reflections

Aww, when I submtted i [rounded] I only got .154ish; ii gave me .151ish. Mind you, this is only based upon 20% of the data, however the question remains - did I use the right metric? Furthermore, the models were only optimized on a raw r2 basis... perhaps things would change if the scoring metric was formerly rmsle.

Additionally, we could still experiment with not natural logging rings, using a fushion of polynomial and log features. Also, we can actually prune the forest (hyperparameter tune it). Furthermore, how about we stop complaining and use SVR???

And finally, for good measure, I'll repeat myself - did I use the right metric?!!?

In [ ]:
#y_pred_clipped = np.clip(y_pred, a_min=0, a_max=None) #Should likely be using this code I found among somebody on Kaggle instead of that loop...

In [ ]:
# log_features = [f for f in num_cols if (train[f] >= 0).all() and scipy.stats.skew(train[f]) > 0]
# log_features

In [9]:
#Some notes in case I need them

In [11]:
# pipe = Pipeline([
#     ('cvec', CountVectorizer(lowercase=False)),
#     ('bc', BaggingClassifier(random_state=26))
# ])
# pipe_params = {'cvec__min_df' : [0.0]
#                ,'cvec__max_df' : [.95]
#                ,'cvec__ngram_range' : [(1,2)]
#                ,'cvec__max_features' : [1800, 1750, 1700]
#                ,'cvec__stop_words' : ['english']
#               }
# gs = GridSearchCV(pipe,
#                   param_grid=pipe_params,
#                   cv=5)

# gs.fit(X_train, y_train)

In [12]:
# print(gs.score(X_train, y_train), gs.score(X_test, y_test))
# print(gs.best_score_)
# print(gs.best_params_)
# #Oh wow, this did a lot 